# LLM Zoomcamp 2024 Competition
This competition requires the use of LLMs to solve high school mathematics problems. The task is to develop models that can accurately solve the problems and then the predictions submitted.

The model in this run is Cohere.

In [1]:
# Import required modules.
import os
import re
import pandas as pd
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
from scorer import score
import cohere

/home/codespace/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/codespace/miniconda3/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [2]:
# Set up Mistral API key.
load_dotenv(Path(find_dotenv()))
api_key = os.getenv("COHERE_API_KEY")

In [3]:
# Define the model.
co = cohere.Client(api_key)

In [4]:
# Function to query the LLM.
def get_answer(question):
    instruction = f'{question} Please return as a 2 decimal place number'
    response = co.chat(
    	message=instruction
    )
    return response.text

In [5]:
def return_answer(text):
    # Use regular expression to find all numbers in the text
    numbers = re.findall(r'-?\d+\.?\d*', text)

    if numbers:
        # Convert the last found number to a float and return it
        return float(numbers[-1])
    else:
        # Return None if no numbers are found
        return None

In [6]:
# Function to prepare the dataset and get answers.
pool = ThreadPoolExecutor(max_workers=6)

def map_progress(pool, seq, f):
    results = []

    with tqdm(total=len(seq)) as progress:
        futures = []

        for el in seq:
            future = pool.submit(f, el)
            future.add_done_callback(lambda p: progress.update())
            futures.append(future)

        for future in futures:
            result = future.result()
            results.append(result)

    return results

In [7]:
# Function for processing each row.
def process_row(row):
    problem_id = row['problem_id']
    problem_text = row['problem_text']

    llm_reasoning = get_answer(problem_text)

    numerical_answer = return_answer(llm_reasoning)

    return {
        'problem_id': problem_id,
        'problem_text': problem_text,
        'llm_reasoning': llm_reasoning,
        'answer': numerical_answer
    }

In [8]:
# Load the train data.
df_train = pd.read_csv('data/train.csv')
df_train.head()

,problem_id,problem_text,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,1.6
1,4723,"In a company of 30 people, 25 use the social n...",24
2,7135,The number of road traffic accidents (RTAs) in...,32
3,5814,Find the value of the expression $\dfrac{2\str...,256
4,9237,A traveler from Moscow wants to visit four cit...,53


In [11]:
# Process row.
rows = df_train.head().to_dict(orient='records')
process_row(rows[0])

{'problem_id': 2374,
 'problem_text': 'Find the value of the expression $\\dfrac{17}{5} :\\dfrac{34}{3} +1.3$.',
 'llm_reasoning': 'The value of the expression is **1.27**.',
 'answer': 1.27}

In [12]:
# Return results df_train head.
results = map_progress(pool, rows, process_row)
df_results = pd.DataFrame(results)
df_results

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.51it/s]


,problem_id,problem_text,llm_reasoning,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,The value of the expression is **1.57**.,1.57
1,4723,"In a company of 30 people, 25 use the social n...",34,34.00
2,7135,The number of road traffic accidents (RTAs) in...,The number of RTAs decreased by **31.82%** in ...,31.82
3,5814,Find the value of the expression $\dfrac{2\str...,The value of the expression is **128.00**.,128.00
4,9237,A traveler from Moscow wants to visit four cit...,125,125.00


 99%|████████████████████████████████████████████████████████████████████████████████████▏| 99/100 [00:09<00:00, 43.25it/s]

In [13]:
# Show df_train head.
df_train.head()

,problem_id,problem_text,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,1.6
1,4723,"In a company of 30 people, 25 use the social n...",24
2,7135,The number of road traffic accidents (RTAs) in...,32
3,5814,Find the value of the expression $\dfrac{2\str...,256
4,9237,A traveler from Moscow wants to visit four cit...,53


In [14]:
# Score df_train head results.
score(df_train.head(), df_results, 'problem_id')

0.0

In [15]:
# Put all together in a function.
def prepare_prompts_and_get_answers(df):
    rows = df.to_dict(orient='records')
    results = map_progress(pool, rows, process_row)
    return pd.DataFrame(results)

In [16]:
# Return the results head.
df_train_results = prepare_prompts_and_get_answers(df_train)
df_train_results.head()

100%|████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:23<00:00,  4.29it/s]


TooManyRequestsError: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 40 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"

In [ ]:
# Get the total score.
score(df_train, df_train_results, 'problem_id')

In [ ]:
# Prepare the test submission.
df_test = pd.read_csv('data/test.csv')

df_test_results = prepare_prompts_and_get_answers(df_test)

submission = df_test_results[['problem_id', 'answer']]
submission.to_csv('submission_mistral', index=False)